```
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```



In [ ]:
#@markdown <h3>← 使用以上代码运行，以防止断开</h>


import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install 

In [ ]:
#@markdown 直接修改config
import copy
import dataclasses
from dataclasses import field, dataclass
from pathlib import Path

import yaml

@dataclass
class Comm:
    main_mode: int = 1
    failed_move: bool = True
    enable_translate: bool = False
    translate_target: str = "zh-cn"
    priority: list = field(default_factory=list)
    cookie: dict = field(default_factory=dict)

    def __post_init__(self):
        self.priority = ["javbus"]


@dataclass
class Resource:
    failed_folder: str = "failed"
    success_folder: str = "output"
    name_rule_folder: str = "number/title"
    name_rule_file: str = "number-title"
    name_max_len: int = 50
    part_suffix: str = ""

    file_type: list = field(default_factory=list)
    exclude_folders: list = field(default_factory=list)
    sub_suffix: list = field(default_factory=list)
    leaked_suffix: list = field(default_factory=list)

    def __post_init__(self):
        self.file_type = [".mp4", ".avi", ".rmvb", ".wmv", ".mov", ".mkv", ".flv", ".ts", ".webm", ".iso"]
        self.exclude_folders = ["failed", "output", "escape"]


@dataclass
class Network:
    enable_proxy: bool = False
    proxy_type: str = "socks5"
    proxy_host: str = ""
    timeout: int = 5
    total: int = 3
    delay: int = 1


@dataclass
class Debug:
    enable: bool = True


@dataclass
class Config:
    comm: Comm = field(default_factory=Comm)
    resource: Resource = field(default_factory=Resource)
    network: Network = field(default_factory=Network)
    debug: Debug = field(default_factory=Debug)

    def default_config(self):
        return copy.deepcopy(self)


_config = Config()
cfg = _config.default_config()

In [ ]:
#@markdown 根据 number 更改优先级
class PriorityQueue:
    def __init__(self):
        self._pq = []
        self._entry_map = {}
        self._counter = itertools.count()
        self.priority = lambda p: -int(p or 0)

    def add(self, task, priority=None):
        if task in self._entry_map:
            old_entry = self._entry_map.pop(task)
            self._pq.remove(old_entry)
        entry = [self.priority(priority), next(self._counter), task]
        self._entry_map[task] = entry
        insort(self._pq, entry)

    def pop(self):
        try:
            return self._pq.pop(0)[-1]
        except IndexError:
            pass

    def empty(self):
        return bool(not self._pq)

    @classmethod
    def get(cls, n: str, website: list):
        pq = cls()
        for w in website:
            pq.add(w, 0)
        # if re.match(r"^\d{5,}", n) or "heyzo" in n.lower():
        #     pq.add("avsox", 1)
        # elif re.match(r"\d+[a-zA-Z]+-\d+", number) or "siro" in number.lower():
        # cls.add("mgstage", 1)
        # elif (re.match(r"\D{2,}00\d{3,}", number) and "-" not in number
        #       and "_" not in number):
        #     cls.add("dmm", 1)
        # elif re.search(r"\D+\.\d{2}\.\d{2}\.\d{2}", number):
        #     cls.add("javdb", 1)
        # elif "fc2" in number.lower():
        #     cls.add("fc2", 1)
        # elif "rj" in number.lower():
        #     cls.add("dlsite", 1)

        return pq


In [ ]:
#@markdown 提取 number

class ExtractNumber:
    _banned = [r"cd\d$", "1080p", "1pon", ".com", "nyap2p", "22-sht.me", "xxx", "carib"]

    def banned(self, name, banned=None):
        banned = self._banned if banned is None else set(banned + self._banned)
        pattern = re.compile(r"\b(" + "|".join(banned) + ")\\W", re.I)
        return pattern.sub("", name).rstrip("-cC")

    @staticmethod
    def west(name):
        if obj := re.search(r"^\D+\d{2}\.\d{2}\.\d{2}", name):
            if sec_obj := re.search(
                    r"^\D+\d{2}\.\d{2}\.\d{2}\.\D+", name, flags=re.I
            ):
                return sec_obj.group()
            return obj.group()

    @staticmethod
    def xxxav(name):
        if obj := re.search(r"XXX-AV-\d{4,}", name.upper(), flags=re.I):
            return obj.group()

    @staticmethod
    def tokyohot(name):
        if obj := re.search(r"n[1|0]\d{3}", name, flags=re.I):
            return obj.group()

    @staticmethod
    def luxu(name):
        if obj := re.search(r"\d{0,3}luxu[-_]\d{4}", name, re.I):
            return obj.group()

    @staticmethod
    def fc2(name):
        if "ppv" in name.lower():
            # if has line, del ppv
            if re.search(r"ppv\s*[-|_]\s*\d{6,}", name, flags=re.I):
                name = re.sub(r"ppv", "", name, flags=re.I)
            # if no line, replace ppv with line
            name = re.sub(
                r"\s{0,2}ppv\s{0,2}", "-", name, flags=re.I
            )
        # if fcxxxx, replace fc with fc2-
        if re.search(r"fc[^2]\d{5,}", name, re.I):
            name = name.replace("fc", "fc2-").replace("FC", "FC2-")
        if obj := re.search(r"fc2[-_]\d{6,}", name, flags=re.I):
            return obj.group()

    @staticmethod
    def regular(name):
        regex = [
            r"[a-z]{2,5}[-_]\d{2,4}",  # bf-123 abp-454 mkbd-120  kmhrs-026
            r"[a-z]{4}[-_][a-z]\d{3}",  # mkbd-s120
            r"\d{6,}[-_][a-z]{4,}",  # 111111-MMMM
            r"\d{6,}[-_]\d{3,}",  # 111111-111
            r"n[-_]*[1|0]\d{3}",  # n1111,n-1111
        ]
        for r in regex:
            if searchobj := re.search(r, name, flags=re.I):
                return searchobj.group()

    @staticmethod
    def no_line(name):
        regex = [
            r"([a-z]{2,5})(\d{2,3})",  # bf123 abp454 mkbd120  kmhrs026
            r"(\d{6,})([a-z]{4,})",  # 111111MMMM
        ]
        for r in regex:
            if obj := re.search(r, name, flags=re.I):
                return obj.group(1) + "-" + obj.group(2)

    def __call__(self, name, *args, **kwargs):
        name = self.banned(name.stem, *args, **kwargs)
        funcs = [self.west(name), self.xxxav(name), self.tokyohot(name), self.luxu(name)]
        for func in funcs:
            if func is not None:
                return func
        if re.search(r'fc.*?\d{5,}', name, flags=re.I):
            return self.fc2(name)
        if "-" in name or "_" in name:
            return self.regular(name)
        return self.no_line(name)

In [ ]:
#@markdown 搜索视频
def search_video(path):
    file_type = cfg.resource.file_type
    excluded = [path.joinpath(e) for e in cfg.resource.exclude_folders]
    files = []
    for file in path.rglob("*"):
        exs = [n for n in excluded if n in file.parents]
        if not exs and file.suffix.lower() in file_type:
            files.append(file)
    return files

def check_folder(path):
    path = Path(path).resolve()
    if path.is_dir():
        files = search_video(path, cfg)
        assert len(files) > 0, 'no video under the folder'
        extract = ExtractNumber()
        number = [extract(f) for f in files]
        assert isinstance(number, list)
        return (path, files, number)


In [ ]:
#@markdown Registry

class Registry:
    _plugins = {}
    _funcs = {}

    @classmethod
    def class_deco(cls, _obj):
        cls._plugins[_obj.__name__] = _obj
        return _obj

    @classmethod
    def func_deco(cls, _func):
        cls._funcs[_func.__qualname__] = _func
        return _func

    @classmethod
    def _register(cls, service_id: str, number):
        service = cls._plugins.get(service_id)
        if service is None:
            raise KeyError(
                "No object named '{}' found in plugins!".format(service)
            )
        return service(number)

    @classmethod
    def get(cls, service_id: str, number):
        obj = cls._register(service_id, number)
        if obj is not None:
            map(exec, [f(obj) for i, f in cls._funcs.items() if service_id in i])
            return obj.data

    @classmethod
    def check_data(cls, service_id, number):
        data = cls.get(service_id, number)
        for key, value in data.items():
            print(f'{key}: {value}')

plug = Registry().class_deco
func = Registry().func_deco


In [ ]:
#@markdown CrawlerComm

# coding: utf-8
import re
import shutil
from collections import defaultdict
from http.cookiejar import LWPCookieJar
from pathlib import Path
from urllib.parse import parse_qs, urlparse, quote_plus
from urllib.request import getproxies

import requests
from requests.adapters import HTTPAdapter, Retry
from requests_html import HTML, HTMLSession

class Metadata(defaultdict):

    def __init__(self):
        super(Metadata, self).__init__(Metadata)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            return ""

    def __setattr__(self, key, value):
        self[key] = value


class RequestHandler:

    def __init__(self, *args, **kwargs):
        """
        Instantiates a new request handler object.
        """
        self._status_forcelist = ["413", "429", "500", "502", "503", "504"]
        self._total = 3
        self._backoff_factor = 1
        self.timeout = cfg.network.timeout
        self.delay = cfg.network.delay
        self.enable_proxy = cfg.network.enable_proxy
        self.proxy_type = cfg.network.proxy_type
        self.proxy_host = cfg.network.proxy_host

    @property
    def proxy_strategy(self):
        if self.enable_proxy and self.proxy_type in ["http", "socks5", "socks5h"]:
            proxy = "{}://{}".format(self.proxy_type, self.proxy_host)
            return {"http": proxy, "https": proxy}
        return getproxies()

    @property
    def retry_strategy(self) -> Retry:
        return Retry(
            total=self._total,
            status_forcelist=self._status_forcelist,
            backoff_factor=self._backoff_factor,
        )

    @property
    def session(self):
        session = HTMLSession()
        adapter = HTTPAdapter(max_retries=self.retry_strategy)
        session.mount("https://", adapter)
        session.mount("http://", adapter)

        assert_status_hook = (lambda response, *args, **kwargs: response.raise_for_status())
        session.hooks["response"] = [assert_status_hook]
        return session

    def get(self, url: str, **kwargs):
        try:
            response = self.session.get(url, timeout=self.timeout, proxies=self.proxy_strategy, **kwargs)
            self.session.close()
            response.encoding = "utf-8"
            return response
        except requests.exceptions.RequestException as exc:
            logger.info(f"RequestError: {exc}")

    def post(self, url, data, **kwargs):
        try:
            response = self.session.post(url, timeout=self.timeout, data=data, proxies=self.proxy_strategy, **kwargs,)
            self.session.close()
            return response
        except requests.exceptions.RequestException as exc:
            logger.info(f"RequestError: {exc}")


class CrawlerBase(RequestHandler):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._data = Metadata()

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, data):
        self._data = data

    def get_parser_html(self, url: str, **kwargs):
        res = self.get(url, **kwargs)
        if res is not None:
            return HTML(html=res.text)

    def default_search(self, number, search_url, waterfall_xpath, id_xpath, url_xpath, **kwargs):
        search_page = self.get_parser_html(search_url, **kwargs)
        parents = search_page.xpath(waterfall_xpath)

        for element in parents:
            num = element.xpath(id_xpath, first=True)
            if num is not None:
                res = re.match("".join(filter(str.isalnum, number)), "".join(filter(str.isalnum, num)), flags=re.I)
                if res is not None:
                    return element.xpath(url_xpath, first=True)

    def get_outline(self, number):
        # jav321
        res = self.post("https://www.jav321.com/search", data={"sn": number})
        if res is not None:
            res.encoding = "utf-8"
            html = HTML(html=res.text)
            outline = html.xpath('//div[@class="panel-body"]/div[@class="row"]/div[@class="col-md-12"]/text()', first=True)
            if outline is not None:
                return outline

        # dmm
        g = GSearch()
        dmm_link = g.search(number, "dmm.co.jp")
        if dmm_link is not None:
            res = self.get_parser_html(dmm_link)
        else:
            # default search
            ...
        if res is not None:
            outline = res.xpath(
                '//div[@class="mg-b20 lh4"]/p[@class="mg-b20"]/text()', first=True
            )
            if outline is not None:
                return outline
        return ""

    def download(self, url, file_name):
        r = self.get(url, stream=True)
        if r.status_code == 200:
            r.raw.decode_content = True

            with open(file_name, "wb") as f:
                shutil.copyfileobj(r.raw, f)
            logger.info(f"successfully download: {file_name}")
        logger.info(f"fail download: {file_name}")

    def download_all(self, img_url: dict, folder):
        for name, url in img_url.items():
            self.download(url, folder.joinpath(name + "jpg"))


class GSearch(RequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        cookie = Path(__file__).parent.parent.joinpath(".google-cookie")
        if cookie.exists():
            self.cookie_jar = LWPCookieJar(str(cookie))
            try:
                self.cookie_jar.load(ignore_discard=True, ignore_expires=True)
            except Exception:
                pass
        else:
            self.cookie_jar = None
            self.get_page("https://www.google.com/")

    def get_page(self, url):
        response = self.session.get(
            url, timeout=self.timeout, proxies=self.proxy_strategy, cookies=self.cookie_jar)
        response.encoding = "utf-8"
        html = response.text
        try:
            self.cookie_jar.save(ignore_discard=True, ignore_expires=True)
        except Exception:
            pass
        return html

    @staticmethod
    def filter(link):
        try:
            if link.startswith("/url?"):
                link = parse_qs(urlparse(link, "http").query)["q"][0]
            url = urlparse(link, "http")
            if url.netloc and "google" not in url.netloc:
                return link
        except Exception:
            ...

    @staticmethod
    def extract(html, number):
        html = HTML(html=html)
        link_content = html.xpath("//a")
        # this list needs to be maintained
        title_xpath = ["//h3/div/text()", "//h3/span/text()"]
        for content in link_content:
            link = content.xpath("//@href", first=True)
            if link is not None:
                if re.search(
                        "".join(filter(str.isalnum, number)),
                        "".join(filter(str.isalnum, link)),
                        flags=re.I,
                ):
                    return link
                # else check title
                for xpath in title_xpath:
                    title = content.xpath(xpath, first=True)
                    if title is None:
                        continue
                    if re.search(
                            "".join(filter(str.isalnum, number)),
                            "".join(filter(str.isalnum, title)),
                            flags=re.I,
                    ):
                        return link

    def search(self, number, site):
        query = quote_plus(number + "+site:" + site)
        html = self.get_page(url=f"https://google.com/search?hl=en&q={query}&safe=off")
        return self.filter(self.extract(html, number))


In [ ]:
#@markdown javubs

import random

@plug
class Javbus(CrawlerBase):
    _url = ["https://www.fanbus.us/", "https://www.javbus.com/", "https://www.busdmm.one/"]

    def __init__(self, number):
        super().__init__()
        base_url = random.choice(self._url)
        self.html = self.get_parser_html(base_url + number)
        # check self.html pass to registry, to decide whether to continue running the following functions
        if self.html is None:
            return

    @func
    def info(self):
        """
        title, number, year, length, actor, director, maker, genre, series
        """
        title = self.html.xpath('//div[@class="container"]/h3/text()', first=True)
        self.data.title = title.lstrip(self.number + "-_ ")
        info = self.html.xpath('//div[@class="col-md-3 info"]', first=True)
        self.data.number = info.xpath('//p[1]/span[2]/text()', first=True)
        self.data.release = info.xpath('//p[2]/text()', first=True)
        self.data.runtime = info.xpath('//p[3]/text()', first=True).replace("分鐘", "")
        self.data.director = info.xpath('//p/span[contains(., "導演")]/../a/text()', first=True)
        self.data.studio = info.xpath('//p/span[contains(., "製作商")]/../a/text()', first=True)
        self.data.publisher = info.xpath('//p/span[contains(., "發行商")]/../a/text()', first=True)

        self.data.tags = info.xpath('//p[position()<last()]/span[@class="genre"]/a/text()')
        self.data.actor = info.xpath('//p[last()]/span[@class="genre"]/a/text()')

        self.data.cover = self.html.xpath('//a[@class="bigImage"]/img/@src', first=True)

        string = self.html.xpath("//a[contains(@class,'sample-box')][1]/@href", first=True)
        if string:
            cid = string.replace('https://pics.dmm.co.jp/digital/video/', '')
            self.data.cid = re.sub('/.*?.jpg', '', cid)

        self.data.outline = self.get_outline(self.number)

In [ ]:
#@markdown javdb

@plug
class Javdb(CrawlerBase, GSearch):
    _url = ["https://javdb.com", "https://javdb4.com", "https://javdb6.com"]

    def __init__(self, number, config):
        super().__init__(config)

        self.base_url = random.choice(self._url)

        # cfg = set_config()
        headers = {
            # "Cookie": cfg.request.cookie.get["javdb"],
            "referer": self.base_url,
        }
        url = self.search(self.number, self.base_url.replace("https://", ""))
        if url is not None:
            self.html = self.get_parser_html(url, headers=headers)
        else:
            self.html = self.search_url(headers)
        if self.html is None:
            return

    def search_url(self, headers):
        search_url = self.base_url + "/search?q=" + self.number + "&f=all"
        xpath = [
            '//div[@class="grid-item column"]',
            '//a/div[@class="uid"]/text()',
            "//a/@href",
        ]
        real_url = self.default_search(
            self.number, search_url, xpath[0], xpath[1], xpath[2], headers=headers
        )
        if real_url is not None:
            return self.get_parser_html(
                self.base_url + real_url, headers=headers
            )

    @func
    def small_cover(self):
        pass

    # def cover(self):
    #     try:
    # self.data.cover = self.html.xpath('//img[@class="video-cover"]/@src')
    # except IndexError:
    #     self.data.cover = ""
    @func
    def outline(self):
        self.data.outline = self.get_outline(self.number)

    @func
    def info(self):
        self.data.title = self.html.xpath('//h2[@class="title is-4"]/strong/text()', first=True)

        info = self.html.xpath('//nav[@class="panel video-panel-info"]', first=True)
        self.data.number = info.xpath("//div[1]/a/@data-clipboard-text", first=True)
        self.data.release = info.xpath('//div/strong[contains(., "日期")]/../span/text()', first=True)
        runtime = info.xpath('//div/strong[contains(., "時長")]/../span/text()', first=True)
        self.data.runtime = runtime.replace("分鍾", "") if runtime is not None else ""
        self.data.director = info.xpath('//div/strong[contains(., "導演")]/../span/a/text()', first=True)
        self.data.studio = info.xpath('//div/strong[contains(., "片商")]/../span/a/text()', first=True)
        self.data.series = info.xpath('//div/strong[contains(., "系列")]/../span/a/text()', first=True)
        self.data.tags = info.xpath('//div/strong[contains(., "類別")]/../span/a/text()')
        self.data.actor = info.xpath('//div/strong[contains(., "演員")]/../span/a/text()')

In [ ]:
#@markdown Comm

# coding: utf-8
import re
import shutil
from pathlib import Path

from lxml.etree import Element, SubElement, ElementTree

def get_metadata(file, number):
    priority = PriorityQueue.get(number, cfg.comm.priority)
    while not priority.empty():
        website = priority.pop().capitalize()
        try:
            data = Registry.get(website, number, cfg)
            if check_data(data):
                return extra_tag(file, data, cfg.resource)
        except Exception as exc:
            logger.info(exc)


def folder_utils(search_path: Path, data, config) -> Path:
    if len(data.actor) >= 5:
        data.actor = data.actor[:2]
    target_path = search_path.joinpath(config.success_folder)
    folder_rule = replace_date(data, config.name_rule_folder)
    level_names = folder_rule.split("/")
    folder = mkdir(target_path.joinpath(level_names[0]))
    for level in level_names[1:]:
        middle = folder.joinpath(level)
        folder = mkdir(middle)
    return folder


def file_utils(created_folder: Path, file: Path, data, config):
    naming_rule = check_length(config.name_rule_file, config.name_max_len)
    filename = replace_date(data, naming_rule)

    filename += ''.join(['-' + i for i in data.extra.values()]) + file.suffix
    return move(file, created_folder.joinpath(filename))


def create_nfo(file_path: Path, data):
    nfo_root = Element("movie")
    filename = file_path.parent.joinpath(file_path.with_suffix(".nfo"))
    nfo_fields = {"title": data.title, "studio": data.studio, "director": data.director, "actor": data.actor,
        "publisher": data.publisher, "runtime": data.runtime, "year": data.year, "tags": data.tags, "cover": data.cover,
    }
    if data.extra.sub:
        nfo_fields["tags"].append("中文字幕")
    if data.extra.leaked:
        nfo_fields["tags"].append("流出")

    for field_name, values in nfo_fields.items():
        if not values:
            continue
        if not isinstance(values, list):
            values = [values]
        for value in values:
            SubElement(nfo_root, field_name).text = f"{value}"

    ElementTree(nfo_root).write(
        str(filename), encoding="utf-8", xml_declaration=True, pretty_print=True
    )


def move(src: Path, dest: Path):
    assert src.exists()
    try:
        shutil.move(src, dest)
        return dest
    except Exception as exc:
        logger.warning(f"fail to move file: {str(exc)}")


def mkdir(dest: Path):
    if dest.exists():
        return dest
    try:
        dest.mkdir(exist_ok=True)
        return dest
    except Exception as exc:
        logger.warning(f"fail to create folder: {str(exc)}")


def create_folder(search_path: Path, folder: str):
    folder = Path(folder).resolve()
    if not folder.is_absolute():
        return mkdir(search_path.parent.joinpath(folder))
    return mkdir(folder)


def replace_date(data, rule: str) -> str:
    replaced = [i for i in data.keys() if i in rule and isinstance(data[i], str)]
    for s in replaced:
        rule = rule.replace(s, data[s].strip())
    return re.sub(r'[?%*:|"<>]', "_", rule)


def check_data(data) -> bool:
    if not data.title or data.title == "null":
        return False
    if not data.number or data.number == "null":
        return False
    return True


def extra_tag(file: Path, data, config):
    filename = file.name

    if any(k in filename.lower() for k in config.leaked_suffix):
        data.extra.leaked = "Leaked"

    if config.part_suffix in filename.lower():
        searchobj = re.search(r"^" + config.part_suffix + r"\d", filename, flags=re.I)
        data.extra.part = searchobj.group() if searchobj else ''

    if any(k in filename.lower() for k in config.sub_suffix):
        data.extra.sub = "C"

    return data


def check_length(name: str, max_len: int) -> str:
    if len(name) > max_len:
        return name[:max_len] if choice.lower() == "y" else check_length(choice, max_len)
    return name


In [ ]:
#@markdown main

path = ""
argument = check_folder(path)

filed_folder = cfg.resource.failed_folder
failed = create_folder(argument[0], filed_folder)
target = dict(zip(argument[1], argument[2]))
for file, number in target.items():
    data = get_metadata(file=file, number=number)
    print(data)
    if not data:
        move(src=file, dest=failed)
        return
    created_folder = folder_utils(argument[0], data, cfg.resource)

    new_file_path = file_utils(created_folder, file, data, cfg.resource)
    # img_utils(created_folder, data, cfg)
    create_nfo(new_file_path, data)